## Interview Query Question #35 | 180 Day Job Postings

Good morning. Here's your sql question for today.

This question was asked by: LinkedIn  
`job_postings` table  

column	type  
id	integer  
job_id	integer  
user_id	integer  
date_posted	datetime  
 

Write a query to return the breakdown of the percentage of users that have posted their jobs either once or within 180 days of posting the same job again.

In [ ]:
# posted once
with posted_once as (
select user_id, job_id, count(*) as num_postings
from job_postings 
group by 1,2
having num_postings = 1
)
,
# posted the same job again within 180 of posting 
posted_180d as (
select 
user_id, 
job_id, 
date_posted, 
lag(date_posted, 1) over (partition by user_id, job_id order by date_posted) as last_posted_at,
date_posted - lag(date_posted, 1) over (partition by user_id, job_id order by date_posted) as diff_days
from job_postings 
where date_posted - lag(date_posted, 1) over (partition by user_id, job_id order by date_posted) <= 180
)
,
# percentage of users breakdown
num_users as (
select count(distinct jp.user_id) as num_all, count(distinct po.user_id) as num_once, count_distinct(p1.user_id) as num_repost
from job_postings jp left join posted_once po on jp.user_id = po.user_id left join posted_180d p1 on jp.user_id = p1.user_id)

select round(num_once / num_all * 100) as pct_once, round(num_once / num_all * 100) as pct_repost
from num_users